In [13]:
import _fixpath

import json
from pathlib import Path
import shutil
from datetime import datetime
import constants


with open(constants.ROOT_DIR / "survey-review-app" / "data" / "possible_detections.json") as f:
    detections = json.load(f)

with open(constants.ROOT_DIR / "survey-review-app" / "data" / "frames.json") as f:
    frames = json.load(f)
frames[0], detections[0]

({'lat': -37.864363521,
  'lon': 175.336249145,
  'alt': 51.4326,
  'gps_time': '2025-08-11T21:22:29.716000+00:00',
  'images': {},
  'line_idx': None},
 {'camera': 'l09',
  'img_path': '2025-08-13/cameras/l09/DCIM/10150813/_09L0183.JPG',
  'detections': [{'x0': 120.07931518554688,
    'y0': 2048.131591796875,
    'x1': 165.4490966796875,
    'y1': 2109.912353515625,
    'label': 'maui-or-hectors',
    'score': 0.5148220062255859,
    'ltwh': [0.01263460807928734,
     0.3232530921396583,
     0.004773756470343079,
     0.009750751533893624]},
   {'x0': 5120,
    'y0': 1624.9957122802734,
    'x1': 5131.011950492859,
    'y1': 1650.07958984375,
    'label': 'maui-or-hectors',
    'score': 0.023116517812013626,
    'ltwh': [0.5387205387205387,
     0.2564702828725179,
     0.0011586648245853206,
     0.003958945322518397]}]})

In [ ]:
for d in detections:
    for f in frames:
        found = False
        for i in f["images"].values():
            if d["img_path"] == i["path"]:
                found = True
                break
        if found:
            d["img"] = i
            d["frame"] = f
            break
    else:
        raise RuntimeError(d["img_path"])
        break


In [28]:
# Edit: used a different dir where I'd manually deleted more, so these don't match
# # Check we didn't miss any
# for res_dir in sorted(
#     Path("/home/ko/customers/maui63/cv/data/inferences/20240815_060150-yolov9-c-20240815-maui/640x640_0.2x0.2/0.01_0.5").iterdir()
# ):
#     rev_dir = res_dir / "reviewed"
#     if not rev_dir.exists():
#         continue
#     for img in rev_dir.rglob("*.jpg"):
#         # Check it's in the detections
#         rel_path = str(img.relative_to(rev_dir))
#         for d in detections:
#             if d['img_path'].lower().endswith(rel_path.lower()):
#                 break
#         else:
#             print(f"Not found: {img}, {rel_path}")

In [ ]:
odir = constants.TMP_DIR / ("to-annotate-" + datetime.now().strftime("%Y%m%d_%H%M%S"))
odir.mkdir()

# Check we didn't miss any
for d in detections:
    img_path = Path("/media/ko/MAUI63/2025-08-maui-winter-survey/") / d["img_path"]
    assert img_path.exists(), img_path
    opath = odir / d["img_path"].replace("/", "_").lower()
    assert not opath.exists(), f"File {opath} already exists"
    print(f"Copying {img_path} to {opath}")
    shutil.copy(img_path, opath)


# for res_dir in sorted(
#     Path("/home/ko/customers/maui63/cv/data/inferences/20240815_060150-yolov9-c-20240815-maui/640x640_0.2x0.2/0.01_0.5").iterdir()
# ):
#     rev_dir = res_dir / "reviewed"
#     if not rev_dir.exists():
#         continue
#     # Copy all images over
#     for img in rev_dir.rglob("*.jpg"):
#         opath = odir / f"{res_dir.name}_{img.name}"
#         assert not opath.exists(), f"File {opath} already exists"
#         print(f"Copying {img} to {opath}")
#         shutil.copy(img, opath)

Copying /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/l09/DCIM/10150813/_09L0183.JPG to /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-13_cameras_l09_dcim_10150813__09l0183.jpg
Copying /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/r09/DCIM/10150813/_09R0863.JPG to /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-13_cameras_r09_dcim_10150813__09r0863.jpg
Copying /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/r09/DCIM/10150813/_09R2920.JPG to /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-13_cameras_r09_dcim_10150813__09r2920.jpg
Copying /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-13/cameras/r28/DCIM/10150813/_28R0077.JPG to /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-13_cameras_r28_dcim_10150813__28r0077.jpg
Copying /media/ko/MAUI63/2025-08-maui-winter-survey/2025-08-14/cameras/l09/DCIM/10050814/_09L5829.JPG to /home/ko/customers/maui63/c

In [ ]:
import exif
import numpy as np
from PIL import Image

# Remove any orientation flags
new_odir = Path(str(odir) + "-no-orientation")
new_odir.mkdir(exist_ok=True)
for img in odir.rglob("*.jpg"):
    im = Image.open(img)
    img_bytes = np.array(im).tobytes()
    im.close()

    with open(img, "rb") as f:
        image = exif.Image(f)

    print(f"Fixing orientation for {img}")
    image.orientation = 1
    new_opath = new_odir / img.name
    with open(new_opath, "wb") as fo:
        fo.write(image.get_file())

    im = Image.open(new_opath)
    new_img_bytes = np.array(im).tobytes()
    im.close()
    assert img_bytes == new_img_bytes, f"Image data changed for {img}"


Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-22_cameras_r09_dcim_10150822__09r2570.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-16_cameras_l09_dcim_10150816__09l0716.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-14_cameras_r09_dcim_10150814__09r3532.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-14_cameras_l28_dcim_10150814__28l6213.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-14_cameras_r09_dcim_10150814__09r0492.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-14_cameras_l28_dcim_10050814__28l5818.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tmp/to-annotate-20251010_163139/2025-08-14_cameras_l28_dcim_10050814__28l3804.jpg
Fixing orientation for /home/ko/customers/maui63/cv/.tm

In [ ]:
assert False, "Go manually annotate with anylabel"

In [ ]:
# Update annotations iwth image metadata:
for d in detections:
    ann_path = new_odir / d["img_path"].replace("/", "_").lower().replace(".jpg", ".json")
    assert ann_path.exists(), f"Annotation file {ann_path} not found"
    with open(ann_path) as f:
        ann = json.load(f)
    ann["src"] = d
    with open(ann_path, "w") as f:
        json.dump(ann, f, indent=2)
    print(ann)

{'version': '0.4.30', 'flags': {}, 'shapes': [{'label': 'maui', 'text': '', 'points': [[4097.169811320755, 2061.7924528301887], [4152.830188679245, 2098.1132075471696]], 'group_id': None, 'shape_type': 'rectangle', 'flags': {}}], 'imagePath': '2025-08-13_cameras_r09_dcim_10150813__09r0863.jpg', 'imageData': None, 'imageHeight': 6336, 'imageWidth': 9504, 'src': {'camera': 'r09', 'img_path': '2025-08-13/cameras/r09/DCIM/10150813/_09R0863.JPG', 'detections': [{'x0': 4101.374755859375, 'y0': 2051.021484375, 'x1': 4152.916015625, 'y1': 2106.080810546875, 'label': 'maui-or-hectors', 'score': 0.10971425473690033, 'ltwh': [0.43154195663503525, 0.32370919892282196, 0.005423112349076705, 0.008689918903389363]}], 'img': {'path': '2025-08-13/cameras/r09/DCIM/10150813/_09R0863.JPG', 't': '2025-08-13T11:32:34.157000+12:00', 'bounds_lbrt': [[174.7277531759015, -37.503970310547444], [174.72552911522905, -37.5042986169796], [174.72501400902019, -37.502083800863524], [174.72723807194805, -37.50175549430